In [ ]:
import pandas as pd
import pyhf 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
MC = pd.read_csv('dataframe.csv')
data = pd.read_csv('Output_ZPrimeBoostedAnalysis/data.csv', delimiter=' ')

In [ ]:
sig = MC.loc[MC['label'] == 1]
bkg = MC.loc[MC['label'] == 0]
sig

In [ ]:
N, bins = np.histogram(data['etmiss'], bins=50, range=(0, 300), weights=data["weight"])
B = np.histogram(bkg['etmiss'], bins=bins, weights=bkg['scaleweight'])[0]
S = np.histogram(sig['etmiss'], bins=bins, weights=sig['scaleweight'])[0]

bin = plt.hist(data['etmiss'], bins=50, weights=data["weight"], range=(0, 300), histtype='step', color='black', label='Data')
plt.hist(bkg['etmiss'], bins=bin[1], weights=bkg['scaleweight'], histtype='step', color='blue', label='Bkg')
plt.hist(sig['etmiss'], bins=bin[1], weights=sig['scaleweight'], histtype='step', color='red', label='Signal')
plt.legend()
plt.show()


In [ ]:
model_spec = {'channels': [{'name': 'singlechannel',
              'samples': [
              {'name': 'signal','data': S.tolist(),
               'modifiers': [{'data': None, 'name': 'mu', 'type': 'normfactor'}]},
              {'name': 'bkg1','data': B.tolist(),
               'modifiers': []},
              ]
              }],
              "observations": [{ "name": "singlechannel", "data": N.tolist() }],
              "measurements": [{ "name": "Measurement", "config": {"poi": "mu", "parameters": []}}],
              "version": "1.0.0",
}

workspace = pyhf.Workspace(model_spec)
model = workspace.model()

print("## Model")
print(f"  channels: {model.config.channels}")
print(f"     nbins: {model.config.channel_nbins}")
print(f"   samples: {model.config.samples}")
print(f" modifiers: {model.config.modifiers}")
print(f"parameters: {model.config.parameters}")
print(f"par. order: {model.config.par_order}")

print("\n## Model parameters")
print(f'   default: {model.config.suggested_init()}')
print(f'    bounds: {model.config.suggested_bounds()}')

data = N.tolist() + model.config.auxdata

test_stat = "qtilde"
test_poi = 1.

# Vary mu values
poi_values = np.linspace(0.01, 5, 500)
obs_limit, exp_limits, scan = pyhf.infer.intervals.upper_limits.upper_limit(data, model, poi_values, level=0.05, return_results=True)
print(f"\nObserved μ upper limit (obs): {obs_limit:.3f}, Expected μ upper limit {exp_limits[2]:.3f}")

CLs_obs, CLs_exp = pyhf.infer.hypotest(test_poi, data, model, test_stat=test_stat, return_expected=True)
print(f"Observed CL_s: {CLs_obs}, Expected CL_s: {CLs_exp} ")

### Dúvidas
- valores dos mu fazem sentido?
- likelihood sem NN
- questão da sensibilidade
- interpretação
- 3 modelos